In [2]:
import numpy as np
import pandas as pd
import xarray as xr
import sys
import os
import io
from typing import *

## Raw data download helper scripts

Helper code for automating download of raw data directly from KNMI. Data must have been recently generated and published on KNMI server.

In [3]:
N_models = 65
tasmin_uri_base = 'https://climexp.knmi.nl/data/icmip5_tasmin_Amon_ens_rcp26_-125--65E_25-50N_n_'
tasmax_uri_base = 'https://climexp.knmi.nl/data/icmip5_tasmax_Amon_ens_rcp26_-125--65E_25-50N_n_'
pr_uri_base = 'https://climexp.knmi.nl/data/icmip5_pr_Amon_ens_rcp26_-125--65E_25-50N_n_'
evspspl_uri_base = 'https://climexp.knmi.nl/data/icmip5_evspsbl_Amon_ens_rcp26_-125--65E_25-50N_n_'
pme_uri_base = 'https://climexp.knmi.nl/data/icmip5_pme_Amon_ens_rcp26_-125--65E_25-50N_n_'
hurs_uri_base = 'https://climexp.knmi.nl/data/icmip5_hurs_Amon_ens_rcp26_-125--65E_25-50N_n_'
psl_uri_base = 'https://climexp.knmi.nl/data/icmip5_psl_Amon_ens_rcp26_-125--65E_25-50N_n_'
for i in range(63, N_models):
    uri = '{}{:03d}.dat'.format(psl_uri_base, i)
    print('Downloading model {}/{} from {}'.format(i+1, N_models, uri))
    !wget $uri

--2019-04-06 21:29:27--  https://climexp.knmi.nl/data/icmip5_psl_Amon_ens_rcp26_-125--65E_25-50N_n_063.dat
Resolving climexp.knmi.nl... 145.23.253.225
Connecting to climexp.knmi.nl|145.23.253.225|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2019-04-06 21:29:29 ERROR 404: Not Found.

--2019-04-06 21:29:29--  https://climexp.knmi.nl/data/icmip5_psl_Amon_ens_rcp26_-125--65E_25-50N_n_064.dat
Resolving climexp.knmi.nl... 145.23.253.225
Connecting to climexp.knmi.nl|145.23.253.225|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2019-04-06 21:29:30 ERROR 404: Not Found.



In [15]:
uri_base = "https://climexp.knmi.nl/data/igridcmip5_tas_Amon_ens_rcp26.3.09a734bba93adc96ea67c85245d72864.info_%LON%_%LAT%_n.dat"
lons = np.arange(181.25, 360.0, 2.5)
lats = np.arange(1.25, 60.0, 2.5)
for lon,lat in zip(lons,lats):
    uri = uri_base.replace("%LON%", '{:07.2f}'.format(lon)).replace("%LAT%", '{:06.2f}'.format(lat))
    print('Downloading data for coordinate {},{} from {}'.format(lon, lat, uri))
    !wget $uri

--2019-04-06 22:15:54--  https://climexp.knmi.nl/data/igridcmip5_tas_Amon_ens_rcp26.3.09a734bba93adc96ea67c85245d72864.info_0181.25_001.25_n.dat
Resolving climexp.knmi.nl... 145.23.253.225
Connecting to climexp.knmi.nl|145.23.253.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44720 (44K) [text/plain]
Saving to: ‘igridcmip5_tas_Amon_ens_rcp26.3.09a734bba93adc96ea67c85245d72864.info_0181.25_001.25_n.dat.1’

igridcmip5_tas_Amon 100%[===================>]  43.67K  71.1KB/s    in 0.6s    

2019-04-06 22:15:56 (71.1 KB/s) - ‘igridcmip5_tas_Amon_ens_rcp26.3.09a734bba93adc96ea67c85245d72864.info_0181.25_001.25_n.dat.1’ saved [44720/44720]

--2019-04-06 22:15:56--  https://climexp.knmi.nl/data/igridcmip5_tas_Amon_ens_rcp26.3.09a734bba93adc96ea67c85245d72864.info_0183.75_003.75_n.dat
Resolving climexp.knmi.nl... 145.23.253.225
Connecting to climexp.knmi.nl|145.23.253.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44720 (44K) [text/pla

Connecting to climexp.knmi.nl|145.23.253.225|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2019-04-06 22:16:13 ERROR 404: Not Found.

--2019-04-06 22:16:13--  https://climexp.knmi.nl/data/igridcmip5_tas_Amon_ens_rcp26.3.09a734bba93adc96ea67c85245d72864.info_0218.75_038.75_n.dat
Resolving climexp.knmi.nl... 145.23.253.225
Connecting to climexp.knmi.nl|145.23.253.225|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2019-04-06 22:16:14 ERROR 404: Not Found.

--2019-04-06 22:16:14--  https://climexp.knmi.nl/data/igridcmip5_tas_Amon_ens_rcp26.3.09a734bba93adc96ea67c85245d72864.info_0221.25_041.25_n.dat
Resolving climexp.knmi.nl... 145.23.253.225
Connecting to climexp.knmi.nl|145.23.253.225|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2019-04-06 22:16:15 ERROR 404: Not Found.

--2019-04-06 22:16:15--  https://climexp.knmi.nl/data/igridcmip5_tas_Amon_ens_rcp26.3.09a734bba93adc96ea67c85245d72864.info_0223.75_043.75_

In [3]:
X_test = xr.open_dataset('data/tas_Amon_CESM2_amip_r1i1p1f1_gn_195001-201412_3.nc', decode_times=False)
print(X_test)

/home/brian/anaconda3/lib/python3.7/site-packages/xarray/conventions.py:394: SerializationWarning: variable 'tas' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  stack_char_dim=stack_char_dim)


<xarray.Dataset>
Dimensions:    (lat: 192, lon: 288, nbnd: 2, time: 780)
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time       (time) float64 7.114e+05 7.114e+05 ... 7.351e+05 7.351e+05
Dimensions without coordinates: nbnd
Data variables:
    lat_bnds   (lat, nbnd) float32 ...
    lon_bnds   (lon, nbnd) float32 ...
    tas        (time, lat, lon) float32 ...
    time_bnds  (time, nbnd) float64 ...
Attributes:
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    case_id:                38
    cesm_casename:          f.e21.FHIST_BGC.f09_f09_mg17.CMIP6-AMIP.001
    contact:                cesm_cmip6@ucar.edu
    creation_date:          2019-01-20T15:59:00Z
    data_specs_version:     01.00.29
    experiment:             AMIP
    experiment_id:          amip
    external_variables:     areacella
    forcing_index:          1
    frequ

## Preprocessing for raw data files

1. Parse and save raw data into per variable datasets

In [3]:
raw_data_dir = 'raw_data'
var_names = ['tas','tasmin','tasmax','pr','pme','evspsbl']
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

In [6]:
import os.path

# create data/ directory
if not os.path.exists('data'):
    os.mkdir('data')

def parse_model_file(filename: str) -> pd.DataFrame:
    def parse_header(lines: List[str]) -> Tuple[Dict[str, str], int]:
        metadata = dict()
        for (i, line) in enumerate(lines):
            # stop at end of header
            if not line.startswith('#'):
                return metadata, i
            # skip header lines that not in key-value format
            if not '::' in line:
                continue
            kv = line.replace('#', '').split('::')
            assert len(kv) == 2
            metadata[kv[0].strip()] = kv[1].strip()
    with open(filename) as f:
        lines = f.readlines()
        metadata, i = parse_header(lines)
        csv_str = "".join(lines[i:])
        df = pd.read_csv(io.StringIO(csv_str), delim_whitespace=True, header=None)
        years = df[0]
        df = df.drop(columns=[0])
        name = '{}_{}'.format(metadata['model_id'], metadata['realization'])
        xdarr = xr.DataArray(df, coords=[years, months], dims=['years', 'months'], attrs=metadata, name=name)
        return xdarr

In [ ]:
for var in var_names:
    dirname = './{}/{}'.format(raw_data_dir, var)
    models = dict()
    for file in filter(lambda f: not f.startswith('.'), os.listdir(dirname)):
        xdarr = parse_model_file('{}/{}'.format(dirname, file))
        models[xdarr.name] = xdarr
    ds = xr.Dataset(models)
    ds.to_netcdf("./data/{}.nc".format(var))

2. Collect and organize raw data to construct per-model datasets

In [4]:
# Utility functions

def center_monthly_means(xdarr: xr.DataArray) -> xr.DataArray:
    attrs = xdarr.attrs
    means = xdarr.mean(dim='years', keep_attrs=True)
    xdarr = xdarr - means
    xdarr.attrs = attrs
    return xdarr

def flatten_months(xdarr: xr.DataArray) -> xr.DataArray:
    d0,d1 = xdarr.shape
    xdarr = xdarr.stack(time=('years','months'))
    assert(len(xdarr.shape) == 1)
    assert(xdarr.shape[0] == d0*d1)
    return xdarr

In [11]:
var_datasets = dict()
for var in var_names:
    xds = xr.open_dataset('data/{}.nc'.format(var))
    var_datasets[var] = xds
    
common_models = set()
for var, ds in var_datasets.items():
    if len(common_models) == 0:
        common_models |= ds.data_vars.keys()
    else:
        common_models &= ds.data_vars.keys()
var_data = dict()
for var, ds in var_datasets.items():
    xs = []
    model_names = []
    print('processing model data for {}'.format(var))
    for model in sorted(filter(lambda m: m in common_models, ds.data_vars.keys())):
        xdarr = ds.data_vars[model]
        # fill NaNs
        xdarr = xdarr.ffill(dim='years')
        xdarr = xdarr.bfill(dim='years')
        xdarr = center_monthly_means(xdarr)
        xdarr = flatten_months(xdarr)
        xs.append(xdarr)
        model_names.append(model)
    print('building data array for {}'.format(var))
    var_dr = xr.DataArray(xs, [('models', model_names),('time', xs[0].indexes['time'])])
    var_data[var] = var_dr

print('building dataset for all variables')
model_time_var_ds = xr.Dataset(var_data)
print(model_time_var_ds)
model_time_var_ds = model_time_var_ds.reset_index('time')
print(model_time_var_ds)
model_time_var_ds.to_netcdf('./data/{}.nc'.format('models_all_vars_vs_time'))

processing model data for tas
building data array for tas
processing model data for tasmin
building data array for tasmin
processing model data for tasmax
building data array for tasmax
processing model data for pr
building data array for pr
processing model data for pme
building data array for pme
processing model data for evspsbl
building data array for evspsbl
building dataset for all variables
<xarray.Dataset>
Dimensions:  (models: 40, time: 2880)
Coordinates:
  * models   (models) <U16 'CCSM4_1' 'CCSM4_2' ... 'MRI-CGCM3_1' 'NorESM1-M_1'
  * time     (time) MultiIndex
  - years    (time) int64 1861 1861 1861 1861 1861 ... 1863 1863 1863 1863 1863
  - months   (time) object 'Jan' 'Feb' 'Mar' 'Apr' ... 'Mar' 'Apr' 'May' 'Jun'
Data variables:
    tas      (models, time) float64 -1.232 -0.9981 -0.3392 ... 1.091 1.176
    tasmin   (models, time) float64 -0.8222 -3.135 -0.5128 ... 1.486 0.5544
    tasmax   (models, time) float64 -0.5729 -2.731 -0.5294 ... 1.633 -0.1004
    pr       (mode

In [5]:
from sklearn.decomposition import PCA
from sklearn.manifold import SpectralEmbedding, TSNE
import matplotlib.pyplot as plt

In [6]:
from dtw import dtw, accelerated_dtw
from typing import Callable
from multiprocessing import Pool
from itertools import product

def _pardtw(params):
    x_i, x_j, metric = params
    d,cost,acc_cost,path = accelerated_dtw(x_i, x_j, metric)
    return d

def pdtw(X, metric: str, verbose: bool = False) -> np.ndarray:
    """
    Returns a function d: X x X -> R that calculates DTW distances from
    a tensor space X, where the second dim of X is time.
    X : data matrix
    metric : metric name to use for DTW (see scipy cdist)
    """
    n, t, m = X.shape
    pool = Pool(4)
    results = pool.map(_pardtw, [(X[i], X[j], metric) for i in range(n) for j in range(n)])
    return np.array(results).reshape((n,n))

In [8]:
ds = xr.open_dataset('data/models_all_vars_vs_time.nc')
X_ds = ds.to_array().transpose('models', 'time', 'variable')
print(X_ds)

<xarray.DataArray (models: 40, time: 2880, variable: 6)>
array([[[ -1.232390e+00,  -8.221712e-01, ...,  -1.762584e-07,  -3.691904e-07],
        [ -9.980754e-01,  -3.134700e+00, ...,  -3.496395e-06,   2.935993e-07],
        ..., 
        [  1.093070e+00,   9.386808e-01, ...,  -3.006784e-06,   2.310148e-06],
        [  1.049638e+00,   1.553081e+00, ...,   5.685991e-06,   2.558547e-06]],

       [[ -6.226550e-01,  -2.455808e-01, ...,   1.469825e-06,   5.744220e-07],
        [ -8.767850e-01,   5.197417e-01, ...,  -7.336508e-06,  -3.874350e-08],
        ..., 
        [  9.840863e-01,   1.147870e+00, ...,  -1.667757e-06,   2.476872e-07],
        [  3.382746e-01,   7.926292e-01, ...,  -5.932489e-06,   3.007603e-06]],

       ..., 
       [[ -1.374901e+00,  -1.929253e+00, ...,   1.195963e-05,  -6.273888e-06],
        [ -5.095788e-01,  -3.281883e+00, ...,   7.633096e-06,   1.095362e-06],
        ..., 
        [  4.450033e-01,   8.567608e-01, ...,  -1.969318e-06,   2.737182e-06],
        [  5.74

In [ ]:
D_x = pdtw(X_ds, 'euclidean')
print(D_x.shape)
print(D_x)
np.save(X_ds, '/data/dtw.npy')